In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt


In [2]:
df = pd.read_excel(r"C:\Users\Minfy\Downloads\Datasets\Bank_Personal_Loan_Modelling.xlsx" , sheet_name= "Data")
df.head()

,ID,Age,Experience,Income,ZIP Code,Family,CCAvg,Education,Mortgage,Personal Loan,Securities Account,CD Account,Online,CreditCard
0,1,25,1,49,91107,4,1.6,1,0,0,1,0,0,0
1,2,45,19,34,90089,3,1.5,1,0,0,1,0,0,0
2,3,39,15,11,94720,1,1.0,1,0,0,0,0,0,0
3,4,35,9,100,94112,1,2.7,2,0,0,0,0,0,0
4,5,35,8,45,91330,4,1.0,2,0,0,0,0,0,1


In [18]:
import mlflow
import mlflow.sklearn

In [ ]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, classification_report

def split_data(df, target_col, test_size=0.3, random_state=42):
    X = df.drop(columns=[target_col])
    y = df[target_col]
    return train_test_split(X, y, test_size=test_size, random_state=random_state, stratify=y)

def create_pipeline():
    pipe = Pipeline([
        ('scaler', StandardScaler()),
        ('rf', RandomForestClassifier(random_state=42))
    ])
    return pipe

def perform_grid_search(pipe, X_train, y_train, param_grid, cv=5, scoring='roc_auc', n_jobs=-1, verbose=1):
    grid_search = GridSearchCV(
        estimator=pipe,
        param_grid=param_grid,
        cv=cv,
        scoring=scoring,
        n_jobs=n_jobs,
        verbose=verbose
    )
    grid_search.fit(X_train, y_train)
    return grid_search

def evaluate_model(model, X_test, y_test):
    y_pred = model.predict(X_test)
    y_proba = model.predict_proba(X_test)[:, 1]

    print("\nTest Set Metrics:")
    print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")
    print(f"Precision: {precision_score(y_test, y_pred):.4f}")
    print(f"Recall: {recall_score(y_test, y_pred):.4f}")
    print(f"F1 Score: {f1_score(y_test, y_pred):.4f}")
    print(f"ROC AUC: {roc_auc_score(y_test, y_proba):.4f}")

    print("\nConfusion Matrix:")
    print(confusion_matrix(y_test, y_pred))

    print("\nClassification Report:")
    print(classification_report(y_test, y_pred))


In [14]:
import mlflow
import mlflow.sklearn

mlflow.set_experiment(experiment_id="989358122986129025")

# Set experiment name (creates it if it doesn't exist)
# mlflow.set_experiment("Loan_Approval_RF_Experiment")

# Enable autologging
mlflow.sklearn.autolog()

if __name__ == "__main__":
    with mlflow.start_run(run_name="RandomForest_GridSearch_Run"):

        # Parameters
        TARGET_COL = 'Personal Loan'
        PARAM_GRID = {
            'rf__n_estimators': [100, 200],
            'rf__max_depth': [None, 10, 20],
            'rf__min_samples_split': [2, 5],
            'rf__min_samples_leaf': [1, 2]
        }

        # Split the data
        X_train, X_test, y_train, y_test = split_data(df, TARGET_COL)

        # Create pipeline
        pipeline = create_pipeline()

        # Grid search
        grid_search = perform_grid_search(pipeline, X_train, y_train, PARAM_GRID)

        print(f"Best Parameters: {grid_search.best_params_}")
        print(f"Best CV AUC: {grid_search.best_score_:.4f}")

        # Evaluate on test set
        evaluate_model(grid_search.best_estimator_, X_test, y_test)

        # Register the best model
        mlflow.sklearn.log_model(
            sk_model=grid_search.best_estimator_,
            artifact_path="best_model",
            registered_model_name="LoanApprovalRandomForest"
        )


2025/07/01 16:10:38 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "c:\Users\Minfy\Desktop\Practice\MLFLOW\.venv\Lib\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details."


Fitting 5 folds for each of 24 candidates, totalling 120 fits


2025/07/01 16:10:58 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "c:\Users\Minfy\Desktop\Practice\MLFLOW\.venv\Lib\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details."
2025/07/01 16:11:06 INFO mlflow.sklearn.utils: Logging the 5 best runs, 19 runs will be omitted.
2025/07/01 16:11:06 WARNING mlflow.utils.

Best Parameters: {'rf__max_depth': 20, 'rf__min_samples_leaf': 1, 'rf__min_samples_split': 2, 'rf__n_estimators': 200}
Best CV AUC: 0.9955

Test Set Metrics:
Accuracy: 0.9873
Precision: 0.9771
Recall: 0.8889
F1 Score: 0.9309
ROC AUC: 0.9983

Confusion Matrix:
[[1353    3]
 [  16  128]]

Classification Report:
              precision    recall  f1-score   support

           0       0.99      1.00      0.99      1356
           1       0.98      0.89      0.93       144

    accuracy                           0.99      1500
   macro avg       0.98      0.94      0.96      1500
weighted avg       0.99      0.99      0.99      1500



2025/07/01 16:11:10 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Registered model 'LoanApprovalRandomForest' already exists. Creating a new version of this model...
Created version '3' of model 'LoanApprovalRandomForest'.


In [ ]:
from sklearn.linear_model import LogisticRegression

# Set experiment name (creates it if it doesn't exist)
# mlflow.set_experiment("Loan_Approval_Logistic_Experiment")

# Enable autologging
mlflow.sklearn.autolog()

def create_logistic_pipeline():
    return Pipeline([
        ('scaler', StandardScaler()),
        ('lr', LogisticRegression(solver='liblinear'))
    ])

LOGISTIC_PARAM_GRID = {
    'lr__C': [0.01, 0.1, 1, 10],
    'lr__penalty': ['l1', 'l2']
}

if __name__ == "__main__":
    with mlflow.start_run(run_name="LogisticRegression_GridSearch_Run"):

        TARGET_COL = 'Personal Loan'

        # Split the data
        X_train, X_test, y_train, y_test = split_data(df, TARGET_COL)

        # Create pipeline
        logistic_pipeline = create_logistic_pipeline()

        # Grid search
        grid_search_logistic = perform_grid_search(
            logistic_pipeline, X_train, y_train, LOGISTIC_PARAM_GRID
        )

        print(f"\n[Logistic Regression] Best Parameters: {grid_search_logistic.best_params_}")
        print(f"[Logistic Regression] Best CV AUC: {grid_search_logistic.best_score_:.4f}")

        # Evaluate
        evaluate_model(grid_search_logistic.best_estimator_, X_test, y_test)

        # Register the best model
        mlflow.sklearn.log_model(
            sk_model=grid_search_logistic.best_estimator_,
            artifact_path="best_logistic_model",
            registered_model_name="LoanApprovalLogisticModel"
        )


2025/07/01 16:03:13 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "c:\Users\Minfy\Desktop\Practice\MLFLOW\.venv\Lib\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details."


Fitting 5 folds for each of 8 candidates, totalling 40 fits


2025/07/01 16:03:14 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "c:\Users\Minfy\Desktop\Practice\MLFLOW\.venv\Lib\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details."
2025/07/01 16:03:23 INFO mlflow.sklearn.utils: Logging the 5 best runs, 3 runs will be omitted.
2025/07/01 16:03:24 WARNING mlflow.utils.a


[Logistic Regression] Best Parameters: {'lr__C': 0.1, 'lr__penalty': 'l2'}
[Logistic Regression] Best CV AUC: 0.9579

Test Set Metrics:
Accuracy: 0.9513
Precision: 0.8318
Recall: 0.6181
F1 Score: 0.7092
ROC AUC: 0.9640

Confusion Matrix:
[[1338   18]
 [  55   89]]

Classification Report:
              precision    recall  f1-score   support

           0       0.96      0.99      0.97      1356
           1       0.83      0.62      0.71       144

    accuracy                           0.95      1500
   macro avg       0.90      0.80      0.84      1500
weighted avg       0.95      0.95      0.95      1500



2025/07/01 16:03:28 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Registered model 'LoanApprovalLogisticModel' already exists. Creating a new version of this model...
Created version '3' of model 'LoanApprovalLogisticModel'.


In [ ]:
from mlflow.tracking import MlflowClient

# Set the experiment name
experiment_name = "Loan_Approval_RF_Experiment"
mlflow.set_experiment(experiment_name)

# Initialize client and get experiment ID
client = MlflowClient()
experiment = client.get_experiment_by_name(experiment_name)
experiment_id = experiment.experiment_id

# Search for the best run (e.g., highest ROC AUC)
runs = client.search_runs(
    experiment_ids=[experiment_id],
    order_by=["metrics.roc_auc DESC"],  # You can change this to another metric if needed
    max_results=1
)

# Get the best run
best_run = runs[0]
best_run_id = best_run.info.run_id
print(f"Best run ID: {best_run_id}")
# print(f"ROC AUC: {best_run.data.metrics['best_cv_score']}")

# Register the model from the best run
model_uri = f"runs:/{best_run_id}/best_model"  # or "best_logistic_model" depending on your artifact path
model_name = "LoanApprovalBestModel_again"

mlflow.register_model(model_uri=model_uri, name=model_name)
print(f"Model registered as: {model_name}")


Successfully registered model 'LoanApprovalBestModel_again'.
2025/07/01 16:14:17 WARNING mlflow.tracking._model_registry.fluent: Run with id 07f5f6890dca46fbb7b8ebcdd99706ee has no artifacts at artifact path 'best_model', registering model based on models:/m-8bbd0956035546588206a8fbefafb989 instead


Best run ID: 07f5f6890dca46fbb7b8ebcdd99706ee
Model registered as: LoanApprovalBestModel_again


Created version '1' of model 'LoanApprovalBestModel_again'.


In [ ]:
import mlflow.pyfunc
import pandas as pd

model_name = "LoanApprovalBestModel"
model_stage = "None"  # or "Staging", "Production", etc.

model_uri = f"models:/{model_name}/latest"  # or specify version like 'models:/LoanApprovalBestModel/1'
model = mlflow.pyfunc.load_model(model_uri)




In [28]:
X_train, X_test, y_train, y_test = split_data(df, TARGET_COL)
predictions = model.predict(X_test)

# View results
print(predictions)

[0 0 0 ... 0 0 0]
